###### This notebook is used for CZ4034's Data Crawling, Classification and file format conversion

# 1.Data Crawling from Trip Advisor

## This section crawls the URL needed to extract the reviews

In [ ]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import pandas as pd
from google.colab import files
print(len(output_links))
#Write to csv
df = pd.DataFrame(list(zip(*[output_links])))
#Add column names
df.columns =["url"]

#Save and export as csv
df.to_csv('URL.csv', index=False, header = False)
files.download('URL.csv')

3244


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def scrape_review(this_url):
  """
    this_url: the url of the website to be scraped on trip advisor

    this function returns a list of objects = [attractionName, reviewerName, reviewTitle, ratings, date, review, reviewerProfileURL]"""


  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  URL = this_url
  #"https://www.tripadvisor.com.sg/ShowUserReviews-g294265-d2149128-r784184084-Gardens_by_the_Bay-Singapore.html"

  try:
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get(URL)  
 
    reviewer_object = driver.find_element_by_xpath("//div[@class='ui_column is-2-desktop is-12-tablet is-12-mobile']") # Find the entire review box
    reviewerName =  reviewer_object.find_element_by_class_name("info_text").text.split("\n")[0]

    reviewerProfileImageObject = driver.find_element_by_xpath(".//div[contains(@class, 'ui_avatar resp')]")
    reviewerProfileImageURL = reviewerProfileImageObject.find_element_by_css_selector("img").get_attribute("src")  

    review_Title = driver.find_element_by_id('HEADING').text #Title of Review e.g Flower bubble in the cloud and rainforest 

    reviews_object = driver.find_elements_by_xpath("//div[@class='ui_column is-10-desktop is-12-tablet is-12-mobile']") # Find the entire review box


    attractionNameObject =  reviews_object[0].find_element_by_xpath(".//div[contains(@class, 'altHeadInline')]")
    attractionName = attractionNameObject.find_element_by_tag_name('a').text

    ratings = reviews_object[0].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")        # ui_bubble_rating bubble_40
    ratings = ratings.split("_")[3] # 40/50


    date = reviews_object[0].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title") #8 March 2021
    review = reviews_object[0].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "") #Cloud forest waterfall was majestic and captivating. You can photo the view from 6th floor,3rd o

    driver.close()

    #print(attractionName) 
    #print(reviewerName)
    #print(review_Title)
    #print(ratings)
    #print(date)
    #print(review)
  except: 
    try:
      driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      driver.get(URL)  
 
      reviewer_object = driver.find_element_by_xpath("//div[@class='ui_column is-2-desktop is-12-tablet is-12-mobile']") # Find the entire review box
      reviewerName =  reviewer_object.find_element_by_class_name("info_text").text.split("\n")[0]

      reviewerProfileImageObject = driver.find_element_by_xpath(".//div[contains(@class, 'ui_avatar resp')]")
      reviewerProfileImageURL = reviewerProfileImageObject.find_element_by_css_selector("img").get_attribute("src")

      review_Title = driver.find_element_by_id('HEADING').text #Title of Review e.g Flower bubble in the cloud and rainforest 

      reviews_object = driver.find_elements_by_xpath("//div[@class='ui_column is-10-desktop is-12-tablet is-12-mobile']") # Find the entire review box
      

      attractionNameObject =  reviews_object[0].find_element_by_xpath(".//div[contains(@class, 'altHeadInline')]")
      attractionName = attractionNameObject.find_element_by_tag_name('a').text

      ratings = reviews_object[0].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")        # ui_bubble_rating bubble_40
      ratings = ratings.split("_")[3] # 40/50


      date = reviews_object[0].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title") #8 March 2021
      review = reviews_object[0].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "") #Cloud forest waterfall was majestic and captivating. You can photo the view from 6th floor,3rd o

      driver.close()
  
    except: 
      attractionName = "error"
      reviewerName = "error"
      review_Title = "error"
      ratings = "error"
      date = "error"
      review = "error"
      reviewerProfileImageURL = "error"


  temp = []
  temp.append(attractionName)
  temp.append(reviewerName)
  temp.append(review_Title)
  temp.append(ratings)
  temp.append(date)
  temp.append(review)
  temp.append(str(reviewerProfileImageURL))


  return temp

In [ ]:
def scrape_list_of_url(this_list):
  """ this_list: input a list of urls to return a list of scraped data """

  output_list = []
  for i in range(0, len(this_list)):
    print("Currently scraping url: " + str(i+1) + " of " + str(len(this_list)))
    scrape_results_list = scrape_review(this_list[i])
    scrape_results_list.append(this_list[i]) #append the url
    output_list.append(scrape_results_list)
  return output_list

## This section crawls the reviews needed to be extracted using the URLs from the previous section

In [ ]:
#Google upload the URL csv file from the 'URL' Folder e.g. ArtScience Museum at Marina Bay Sands URL

import pandas as pd
from google.colab import files

uploaded_url = files.upload()
filenames_url = uploaded_url.keys()

filenames_url_list = []

for file in filenames_url:
    filenames_url_list.append(file)

print(filenames_url_list)

Saving Gardens by the bay URL.csv to Gardens by the bay URL.csv
['Gardens by the bay URL.csv']


In [ ]:
#Load the URL csv
for filename in filenames_url_list:
    if filename.endswith(".csv"):
      df = pd.read_csv(filename, header = None)
      url_list = df.values.tolist()
      templist = []
      for url in range(0, len(url_list)):
        templist.append(url_list[url][0])

print(len(templist))
#print(templist)

3244


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
scraped_list_results = scrape_list_of_url(templist)
#print(scraped_list_results)

In [ ]:
results_df = pd.DataFrame.from_records(scraped_list_results)
results_df.columns =["attractionName","reviewerName","reviewTitle","ratings", "reviewDate", "reviewText","reviewerProfilePicURL", "reviewURL"]

logfile_name = "results.csv"
results_df.to_csv(logfile_name, index = False)
files.download(logfile_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Scrape Profile Pictures Only (Deprecated - skip)

this section was used to scrape profile pictures url on a custom basis
and is now part of the url_scrape

In [ ]:
def scrape_review2(this_url):
  """
    this_url: the url of the website to be scraped on trip advisor

    this function returns a list of objects = [attractionName, reviewerName, reviewTitle, ratings, date, review, reviewerProfileURL]"""


  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  URL = this_url
  #"https://www.tripadvisor.com.sg/ShowUserReviews-g294265-d2149128-r784184084-Gardens_by_the_Bay-Singapore.html"

  try:
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get(URL)  
 

    reviewerProfileImageObject = driver.find_element_by_xpath(".//div[contains(@class, 'ui_avatar resp')]")
    reviewerProfileImageURL = reviewerProfileImageObject.find_element_by_css_selector("img").get_attribute("src")  

    driver.close()
    
  except: 
    try:
      driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      driver.get(URL)  
 
      reviewerProfileImageObject = driver.find_element_by_xpath(".//div[contains(@class, 'ui_avatar resp')]")
      reviewerProfileImageURL = reviewerProfileImageObject.find_element_by_css_selector("img").get_attribute("src")  

      driver.close()
  
    except: 
      reviewerProfileImageURL = "error"


  temp = []
  temp.append(str(reviewerProfileImageURL))

  return temp

In [ ]:
def scrape_list_of_url2(this_list):
  """ this_list: input a list of urls to return a list of scraped data """

  output_list = []
  for i in range(0, len(this_list)):
    print("Currently scraping url: " + str(i+1) + " of " + str(len(this_list)))
    scrape_results_list = scrape_review2(this_list[i])
    scrape_results_list.append(this_list[i]) #append the url
    output_list.append(scrape_results_list)
  return output_list

In [ ]:
#Google upload the URL csv file

import pandas as pd
from google.colab import files

uploaded_url = files.upload()
filenames_url = uploaded_url.keys()

filenames_url_list = []

for file in filenames_url:
    filenames_url_list.append(file)

print(filenames_url_list)

Saving cb2.csv to cb2.csv
['cb2.csv']


In [ ]:
#Load the URL csv
for filename in filenames_url_list:
    if filename.endswith(".csv"):
      df = pd.read_csv(filename, header = None)
      url_list = df.values.tolist()
      templist = []
      for url in range(0, len(url_list)):
        templist.append(url_list[url][0])

print(len(templist))
#print(templist)

2165


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
scraped_list_results = scrape_list_of_url2(templist)

In [ ]:
results_df = pd.DataFrame.from_records(scraped_list_results)
results_df.columns =["reviewerProfilePicURL", "reviewURL"]

logfile_name = "results.csv"
results_df.to_csv(logfile_name, index = False)
files.download(logfile_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2.VADER Sentiments Analysis
## This section conducts sentiment analysis using Google Translator and VADER

In [ ]:
#Google upload the reviewtext file 'ReviewsText Only' from the 'Supporting Document' Folder

import pandas as pd
from google.colab import files

uploaded_url = files.upload()
filenames_url = uploaded_url.keys()

filenames_url_list = []

for file in filenames_url:
    filenames_url_list.append(file)

print(filenames_url_list)

Saving Combined Data ReviewsText Only.csv to Combined Data ReviewsText Only.csv
['Combined Data ReviewsText Only.csv']


In [ ]:
#Load the URL csv
import csv
for filename in filenames_url_list:
    if filename.endswith(".csv"):
      with open(filename, newline='', encoding="ISO-8859-1") as f:
        reader = csv.reader(f)
        reviews_data = list(reader)
  
#print(reviews_data[0])
#print(reviews_data[1][0])

["It is a unique architectural form stands out as an icon within the equally striking Marina Bay Sands which is one of the two integrated resorts in Singapore. While the other focuses on fun, MBS focus on exhibitions and conventions.The exterior form while viewed from far is liken the splashing of water and has relevance to the ship like form of the hotel behind. When you draw near, you will noticed that the building is supported mainly by concrete (I think) stilts, given a sense of impossible support. To maintain this openness feel, the entrance to the museum is encased in glass and it's so well done that i have never noticed it the last 5 or so years I have been passing in front of it, unless I have a reason to look for it today.Beyond the entrance lies a cafeteria which is well positioned to serve as a waiting area for visitors to the museum. The exhibition halls are located above and below this level and is reached by lifts and stairs (I think).At the basement, the middle opens int

In [ ]:
!pip install vaderSentiment
!pip install googletrans-temp

In [ ]:
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


def sentiment_scores(sentence): 

    #Translate to English
    print("OrgSentence: ", sentence)

    try:
      translator = Translator()
      sentence = translator.translate(sentence).text
      #print("Translated Sentence: ", sentence)
    except:
      None
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
    #print("Overall sentiment dictionary is : ", sentiment_dict) 
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
  
    #print("Sentence Overall Rated As", end = " ") 
    sentiment_category = "Neutral"

    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        #print("Positive") 
        sentiment_category = "Positive"
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        #print("Negative") 
        sentiment_category = "Negative"
  
    else : 
        #print("Neutral")
        sentiment_category = "Neutral"
    
    return sentiment_category, sentiment_dict['compound']

#str_a = "You are terrible!!"
#print(sentiment_scores(str_a))
#('Negative', -0.5399)

In [ ]:
review_output_list = []

for i in range(0,len(reviews_data)): 
  comments = reviews_data[i][0]

  try:
    sentiment_category, compound_score = sentiment_scores(comments)
    print("Row: ", i, " ", compound_score, ",", sentiment_category)
  
  except:
    try:
    #Try again if fail
      sentiment_category, compound_score = sentiment_scores(comments)
      print("Row: ", i, " ", compound_score, ",", sentiment_category)
    
    except:
      sentiment_category, compound_score = sentiment_scores(comments)
      print("Row: ", i, " ", compound_score, ",", sentiment_category)


  templist = reviews_data[i][:]
  templist.append(compound_score)
  templist.append(sentiment_category)
  #print(templist)

  review_output_list.append(templist)

In [ ]:
#Write to csv. Output can be found in the 'VADER Sentiments Output' Folder
import pandas

df = pandas.DataFrame(review_output_list,columns=['reviewText','compound_score', 'sentiment_category'])
df.to_csv('review_VaderSentiments.csv', index= False) 
files.download('review_VaderSentiments.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 3.Classification Modelling

In [ ]:
import nltk
import unicodedata
import re
from google.colab import files
import csv

from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Google upload the labeled data csv file 'Combined Data -Ten Percent Labeled - 1025'

uploaded_labeled_data = files.upload()
filenames_label = uploaded_labeled_data.keys()

filenames_label_list = []

for file in filenames_label:
    filenames_label_list.append(file)

print(filenames_label_list)

Saving Combined Data - Labels - 1025 Tenpercent.csv to Combined Data - Labels - 1025 Tenpercent.csv
['Combined Data - Labels - 1025 Tenpercent.csv']


In [ ]:
#Read in 10% of labeled data
for filename in filenames_label_list:
    if filename.endswith(".csv"):
      df=pd.read_csv(filename, encoding="ISO-8859-1")
#df.head()
print(len(df))

1025


In [ ]:
#Google upload the 100% data csv file 'Combined Data - Raw Trip Advisor Reviews'

uploaded_unlabeled_data = files.upload()
filenames_unlabel = uploaded_unlabeled_data.keys()

filenames_unlabel_list = []

for file in filenames_unlabel:
    filenames_unlabel_list.append(file)

print(filenames_unlabel_list)

Saving Combined Data.csv to Combined Data.csv
['Combined Data.csv']


In [ ]:
#Read in 100% of 10K data
for filename in filenames_unlabel_list:
    if filename.endswith(".csv"):
      whole_df=pd.read_csv(filename, encoding="ISO-8859-1")
#whole_df.head()
print(len(whole_df))

10114


In [ ]:
#Data preprocessing
for corpus in ["stopwords", "punkt", "averaged_perceptron_tagger"]:
        check_corpus(corpus)

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

def remove_stopwords(text, is_lower_case=False):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
  
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def normalize_corpus(doc,remove_digits=False):
    doc_ac = remove_accented_chars(doc)
    doc = doc_ac.lower()
    doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
    special_char_pattern = re.compile(r'([{.(-)!}])')
    doc = special_char_pattern.sub(" \\1 ", doc)
    doc = remove_special_characters(doc, remove_digits=remove_digits)  
    doc = re.sub(' +', ' ', doc)
    doc = remove_stopwords(doc)
    return doc

In [ ]:
import spacy
from collections import Counter
import json
nlp = spacy.load('en_core_web_sm')
from itertools import groupby
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import Tree
from nltk import pos_tag
from nltk.chunk import RegexpParser
from nltk import ne_chunk
import copy
from sklearn import preprocessing

#Defining functions
def check_corpus(corpus_name):
    """
    This function checks whether a corpus in nltk is installed. If not available, it will automatically download.
    :param corpus_name: name of the corpus
    """
    try:
        nltk.data.find('corpus_name')
        print("{} is installed".format(corpus_name))
    except LookupError:
        print("{} is not installed. Trying to download now...".format(corpus_name))
        nltk.download(corpus_name)

def tokenizer_master(sentence):
    sentss = sent_tokenize(sentence.lower())
    sentss2 = []
    for each_sent in sentss:
        sentss2.extend([word_tokenize(each_sent)])
    return sentss2

def tagged_sentences(document):
    tagged_sentences = [pos_tag(sent) for sent in document]
    return tagged_sentences

def get_chunks(tagged_sentences):
    master_list = []
    master_noun = []
    master_adj = []
    grammar = r"""
    CHUNK1:
        {<NN.*><.*>{0,3}<JJ.*>}  # Any Noun terminated with Any Adjective
    
    CHUNK2:
        {<JJ.*><.*>{0,3}<NN.*>}  # Nouns or Adjectives, terminated with Nouns
    """
    cp = RegexpParser(grammar)
    for sent in tagged_sentences:
        tree = cp.parse(sent)
        for subtree in tree.subtrees(filter = lambda t: t.label() in ['CHUNK1', 'CHUNK2']):
            if (str(subtree).find('NN') >0 or str(subtree).find('NNS') >0 or str(subtree).find('NNP') >0) and (str(subtree).find('JJ')>0 or str(subtree).find('JJS')>0 or str(subtree).find('JJR')>0):
                nouns = [word for word, tag in subtree.leaves() if tag in ['NN', 'NNS','NNP']]
                adjss = [word for word, tag in subtree.leaves() if tag in ['JJ','JJR','JJS']]
                master_noun.extend([nouns])
                master_adj.extend([adjss])

    return [m[0]+":"+n[0] for m,n in zip(master_noun,master_adj)]

def get_chunks_secondary(tagged_sentences):
    master_list = []
    master_noun = []
    master_adj = []
    grammar = r"""
    CHUNK1:
        {<NN.*><.*>{0,3}<JJ.*>}  # Any Noun terminated with Any Adjective
    
    CHUNK2:
        {<JJ.*><.*>{0,3}<NN.*>}  # Nouns or Adjectives, terminated with Nouns
    """
    cp = RegexpParser(grammar)
    for sent in tagged_sentences:
        tree = cp.parse(sent)
        for subtree in tree.subtrees(filter = lambda t: t.label() in ['CHUNK1', 'CHUNK2']):
            if (str(subtree).find('NN') >0 or str(subtree).find('NNS') >0 or str(subtree).find('NNP') >0) and (str(subtree).find('JJ')>0 or str(subtree).find('JJS')>0 or str(subtree).find('JJR')>0):
                nouns = [word for word, tag in subtree.leaves() if tag in ['NN', 'NNS','NNP']]
                adjss = [word for word, tag in subtree.leaves() if tag in ['JJ','JJR','JJS']]
                master_noun.extend([nouns])
                master_adj.extend([adjss])

    return [m[0]+":"+n[0] for m,n in zip(master_noun,master_adj)], master_noun, master_adj

In [ ]:
aspectCountList = []
for review in df['reviewText']:
  aspectCountList.append(len(get_chunks(tagged_sentences(tokenizer_master(remove_stopwords(remove_special_characters(review)))))))

df['aspect count']= aspectCountList
df.head()

,attractionName,reviewerName,reviewTitle,ratings,reviewDate,reviewText,reviewerProfilePicURL,reviewURL,compound_score,sentiment_category,wordCount,usefulness,aspect count
0,Gardens by the Bay,631jayshreem,Gardens by the Bay,40,11-Nov-19,Marvellous Human creation....The flowers..The ...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.6249,Positive,12,useful,2
1,Jurong Bird Park,cris t,"excellant park, its clean",40,14-Jan-18,"like everything in Singapore, its done right. ...",https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.7845,Positive,12,useful,2
2,Gardens by the Bay,ANJ002,Pricey entrance into sheltered gardens,30,10-Dec-14,Like: Chance to learn different kinds of veget...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.5423,Positive,13,useful,2
3,National Orchid Garden,Alnobbi,the richest Orchid Garden I've ever seen,50,22-Mar-20,an uncomparable experience: just wander around...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.7264,Positive,14,useful,3
4,Singapore Cable Car,Siana2011,Boring Ride,20,28-Dec-11,A boring ride. Not much scenery to enjoy. Rela...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.1346,Positive,14,useful,3


In [ ]:
corpus = [normalize_corpus(whole_df['reviewText'][i]) for i in range(whole_df.shape[0])]
vectorizer = TfidfVectorizer(max_df=0.5, stop_words='english', use_idf=True)
X = vectorizer.fit_transform(corpus).todense()
Xdf=pd.DataFrame(X)

df_merged=pd.merge(df, Xdf, left_index=True, right_index=True)
df_merged.head(3)

,attractionName,reviewerName,reviewTitle,ratings,reviewDate,reviewText,reviewerProfilePicURL,reviewURL,compound_score,sentiment_category,wordCount,usefulness,aspect count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,17893,17894,17895,17896,17897,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932
0,Gardens by the Bay,631jayshreem,Gardens by the Bay,40,11-Nov-19,Marvellous Human creation....The flowers..The ...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.6249,Positive,12,useful,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Jurong Bird Park,cris t,"excellant park, its clean",40,14-Jan-18,"like everything in Singapore, its done right. ...",https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.7845,Positive,12,useful,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Gardens by the Bay,ANJ002,Pricey entrance into sheltered gardens,30,10-Dec-14,Like: Chance to learn different kinds of veget...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.5423,Positive,13,useful,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
trainDF = df_merged.copy()

input_exclude=['attractionName','reviewerName','reviewTitle','reviewDate','reviewText','reviewerProfilePicURL','reviewURL','sentiment_category','usefulness']
trainDF = trainDF.drop(input_exclude,axis=1)

x = trainDF.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
trainDF = pd.DataFrame(x_scaled)



trainDF['usefulness'] = df_merged['usefulness']
trainDF['usefulness'] = trainDF['usefulness'].map({'useful': 1, 'not useful': 0})
#trainDF.rename({'0': 'ratings', '1': 'compoundScore','2': 'wordCount','3': 'aspectCount'}, axis=1, inplace = True)
trainDF.head()



train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF, trainDF['usefulness'], random_state= 22, train_size = 0.8)

print(train_x.shape, valid_x.shape, train_y.shape, valid_y.shape)
print("Train X, Test X, Train Y, Test Y")

(820, 17938) (205, 17938) (820,) (205,)
Train X, Test X, Train Y, Test Y


In [ ]:
# helper function
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

global pred
pred=[]
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    pred.append(predictions)  
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        # print('tn,fp-> supposed to be 0 but we predict 1'
        #       'fn,tp')
        # we want to lower fn-> we want to increase recall.
    print(confusion_matrix(predictions, valid_y))
    print("Recall: ",recall_score(predictions, valid_y))
    print("Precision: ",precision_score(predictions, valid_y))
    print('F1: ', f1_score(predictions, valid_y))
    print('Accuracy: ', metrics.accuracy_score(predictions, valid_y))
    return classifier

In [ ]:
print('==Naive Bayes==')
from sklearn import naive_bayes
nb = train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x)

#[[ 13   0]
# [ 46 146]]

#Recall: 0.760416667
#Precision:  1
#F1: 0.863905325
#Accuracy: 0.775609756
#"""

===Naive Bayes===
[[ 13   0]
 [ 46 146]]
Recall:  0.7604166666666666
Precision:  1.0
F1:  0.8639053254437871
Accuracy:  0.775609756097561


In [ ]:
valid_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932,17933,17934,17935,17936,usefulness
1007,0.50,0.961340,0.254206,0.260870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
337,0.25,0.293833,0.050467,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
261,0.50,0.513879,0.048598,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
242,0.50,0.219945,0.039252,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
999,1.00,0.834057,0.177570,0.130435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
train_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932,17933,17934,17935,17936,usefulness
341,0.50,0.586404,0.057944,0.086957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
400,0.25,0.395629,0.065421,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
441,0.75,0.942919,0.287850,0.217391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
332,0.50,0.957757,0.056075,0.065217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
33,0.50,0.864591,0.029907,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
print('==Logistic Regressionr==')
from sklearn import linear_model
lm = train_model(linear_model.LogisticRegression(), train_x, train_y, valid_x)

#[[ 37   9]
# [ 22 137]]

#Recall: 0.86163522
#Precision:  0.938356164
#F1: 0.898360656
#Accuracy: 0.848780488
#"""

In [ ]:
print('==SVM==')
from sklearn import svm
svm = train_model(svm.SVC(), train_x, train_y, valid_x)

#[[ 45   1]
# [ 14 145]]

#Recall: 0.911949686
#Precision:  0.993150685
#F1: 0.950819672
#Accuracy: 0.926829268
#"""

In [ ]:
print('==Random Forest==')
from sklearn import ensemble
rf = train_model(ensemble.RandomForestClassifier(), train_x, train_y, valid_x)

#[[ 49   3]
# [ 10 143]]

#Recall: 0.934640523
#Precision:  0.979452055
#F1: 0.956521739
#Accuracy: 0.936585366
#"""

In [ ]:
print('==Xgboost==')
import xgboost
xgb = train_model(xgboost.XGBClassifier(), train_x, train_y, valid_x)

#Results

#[[ 51   4]
# [  8 142]]

#Recall: 0.946666667
#Precision:  0.97260274
#F1: 0.959459459
#Accuracy: 0.941463415
#"""

In [ ]:
final_pred=[]
weight=[0.8,1.5,1.5,1.5,0.8]
for i in range(len(valid_y)):
  lst=0
  count=0
  for ary in range(len(pred)):
    lst=(weight[count])*(lst+(pred[ary][i]))
    count+=1
  if lst>=7.8:
    final_pred.append(1)
  else:
    final_pred.append(0)
print(confusion_matrix(final_pred, valid_y))
print("Recall: ",recall_score(final_pred, valid_y))
print("Precision: ",precision_score(final_pred, valid_y))
print('F1: ', f1_score(final_pred, valid_y))
print('Accuracy: ', metrics.accuracy_score(final_pred, valid_y))


#Results

#weight=[0.8,0.8,1.3,1.5,1.5]
#[[ 46   1]
# [  13 145]]
#Recall:  0.917721518987342
#Precision:  0.993150684931507
#F1:  0.953947368421053
#Accuracy:  0.931707317073171
#"""

In [ ]:
# save the model to disk
#filename = 'finalized_model.sav'
#joblib.dump(model, filename)

# load the model from disk
#loaded_model = joblib.load(filename)
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [ ]:
aspectCountList = []
for review in whole_df['reviewText']:
  aspectCountList.append(len(get_chunks(tagged_sentences(tokenizer_master(remove_stopwords(remove_special_characters(review)))))))

whole_df['aspect count']= aspectCountList
whole_df.head()

,attractionName,reviewerName,reviewTitle,ratings,reviewDate,reviewText,reviewerProfilePicURL,reviewURL,compound_score,sentiment_category,wordCount,aspect count
0,ArtScience Museum at Marina Bay Sands,KYHENG,Amazing architecture / interesting exhibition,50,13-Mar-21,It is a unique architectural form stands out a...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.9798,Positive,285,25
1,ArtScience Museum at Marina Bay Sands,Sandra J,Surprisingly kids friendly - a hit!,50,17-Feb-21,Went to see 'Futureworld: Where Art Meets Scie...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.9873,Positive,54,4
2,ArtScience Museum at Marina Bay Sands,Lovetrip1027,Team Lab Amazing Artwork!,50,13-Feb-21,Always wanted to visit their exhibit! Their di...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.8607,Positive,53,5
3,ArtScience Museum at Marina Bay Sands,imthecraziestdude,Amazing pictures! must seeing destinations in ...,50,9-Sep-20,I love that place since Im a big art fan! And ...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.9603,Positive,65,7
4,ArtScience Museum at Marina Bay Sands,NellyC76,Great for kids and adults alike!,50,29-Jun-20,We visited two exhibitions here - Disney and F...,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com.sg/ShowUserReviews...,0.4440,Positive,77,6


In [ ]:
aspectCount_df = whole_df[['aspect count']]
aspectCount_df.to_csv('aspect count.csv', index = False)
files.download('aspect count.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
input_exclude=['attractionName','reviewerName','reviewTitle','reviewDate','reviewText','reviewerProfilePicURL','reviewURL','sentiment_category']
whole_df = whole_df.drop(input_exclude,axis=1)
whole_df.head()

,ratings,compound_score,wordCount,aspect count
0,50,0.9798,285,25
1,50,0.9873,54,4
2,50,0.8607,53,5
3,50,0.9603,65,7
4,50,0.4440,77,6


In [ ]:
whole_x = whole_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
whole_x_scaled = min_max_scaler.fit_transform(whole_x)
whole_df = pd.DataFrame(whole_x_scaled)
whole_df.head

<bound method NDFrame.head of          0         1         2         3
0      1.0  0.990068  0.257713  0.271739
1      1.0  0.993830  0.048094  0.043478
2      1.0  0.930327  0.047187  0.054348
3      1.0  0.980287  0.058076  0.076087
4      1.0  0.721308  0.068966  0.065217
...    ...       ...       ...       ...
10109  0.0  0.691964  0.046279  0.054348
10110  0.0  0.068569  0.039927  0.032609
10111  0.0  0.799709  0.085299  0.086957
10112  0.0  0.413473  0.062613  0.076087
10113  0.0  0.799258  0.026316  0.021739

[10114 rows x 4 columns]>

In [ ]:
whole_df_merged=pd.merge(whole_df, Xdf, left_index=True, right_index=True)
whole_df_merged.head(3)

,0_x,1_x,2_x,3_x,0_y,1_y,2_y,3_y,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,17893,17894,17895,17896,17897,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932
0,1.0,0.990068,0.257713,0.271739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.993830,0.048094,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.930327,0.047187,0.054348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
whole_df_merged['usefulness'] = 0

In [ ]:
whole_df_merged.head(3)

,0_x,1_x,2_x,3_x,0_y,1_y,2_y,3_y,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,17894,17895,17896,17897,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932,usefulness
0,1.0,0.990068,0.257713,0.271739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.993830,0.048094,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.930327,0.047187,0.054348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
col_names = list(range(0, 17937))
col_names.append('usefulness')
print(len(col_names))

whole_df_merged.columns = col_names
whole_df_merged.head(3)

17938


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,17898,17899,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919,17920,17921,17922,17923,17924,17925,17926,17927,17928,17929,17930,17931,17932,17933,17934,17935,17936,usefulness
0,1.0,0.990068,0.257713,0.271739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.993830,0.048094,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.930327,0.047187,0.054348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
from datetime import datetime
print('Start time:' ,datetime.now())

whole_pred=[]
whole_pred.append(nb.predict(whole_df_merged))
whole_pred.append(lm.predict(whole_df_merged))
whole_pred.append(svm.predict(whole_df_merged))
whole_pred.append(rf.predict(whole_df_merged))
whole_pred.append(xgb.predict(whole_df_merged))

weight=[0.8,0.8,1.3,1.5,1.5]
for i in range(len(whole_df_merged)):
  lst=0
  count=0
  for ary in range(len(whole_pred)):
    lst=(weight[count])*(lst+(whole_pred[ary][i]))
    count+=1
  print(lst)
  if lst>=7.8:
    whole_df_merged.at[i,'usefulness']=1
  else:
    whole_df_merged.at[i,'usefulness']=0
print('End time:' ,datetime.now())

Start time: 2021-04-05 14:28:15.607830
End time: 2021-04-05 14:30:50.988299


In [ ]:
whole_df_merged.head()
udf = whole_df_merged[['usefulness']]
udf.head()
udf.to_csv('finalout.csv', index = False)
files.download('finalout.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
whole_df_merged.to_csv('finalout.csv', index = False)
files.download('finalout.csv')

# 4.Convert CSV to XML
## This section is used to conver the dataset from csv format to XML such that the data can be fed into SOLR

In [ ]:
import pandas as pd
import csv
from google.colab import files

#Upload XML Sample.csv in the 'SOLR XML' Folder

filename = "XML Sample.csv"
with open(filename, newline='', encoding="ISO-8859-1") as csv_to_xml:
  reader = csv.reader(csv_to_xml)
  data_list = list(reader)

print(data_list[0])
print(len(data_list))
#['attractionName', 'reviewerName', 'reviewTitle', 'ratings', 'reviewDate', 'reviewText', 'reviewerProfilePicURL', 'reviewURL', 'compound_score', 'sentiment_category', 'synonyms','usefulness','usefulness_score']

In [ ]:
def convert_month(month):
  month = month.lower()
  if(month == "jan"):
    return str("01")
  if(month == "feb"):
    return str("02")
  if(month == "mar"):
    return str("03")
  if(month == "apr"):
    return str("04")
  if(month == "may"):
    return str("05")
  if(month == "jun"):
    return str("06")
  if(month == "jul"):
    return str("07")
  if(month == "aug"):
    return str("08")
  if(month == "sep"):
    return str("09")
  if(month == "oct"):
    return str("10")
  if(month == "nov"):
    return str("11")
  if(month == "dec"):
    return str("12")
  else:
    return str("01")

def convert_date(date):
  datelist = (date.split("-"))
  day = datelist[0]
  month = convert_month(datelist[1])
  year = "20" + str(datelist[2])
  newdate = str(year + "-" + month + "-" + day)
  return(newdate)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def convert_row(row):
    """Support function to convert each row into XML format
    """

    header_list = ['attractionName', 'reviewerName', 'reviewTitle', 'ratings', 'reviewDate', 'reviewText', #0-5
                   'reviewerProfilePicURL', 'reviewURL', 'compoundScore', 'sentimentCategory','synonyms',  #6-10
                   'usefulness', 'usefulness_score']

    element0 = row[0]
    element1 = row[1]
    element2 = row[2]
    element3 = row[3]
    element4 = convert_date(row[4])
    element5 = row[5]
    element6 = row[6]
    element7 = row[7] 
    element8 = row[8]
    element9 = row[9]
    element10 = row[10]
    element11 = row[11]
    element12 = row[12]

    header0= header_list[0]
    header1= header_list[1]
    header2= header_list[2]
    header3= header_list[3]
    header4= header_list[4]
    header5= header_list[5]
    header6= header_list[6]
    header7= header_list[7]
    header8= header_list[8]
    header9= header_list[9]
    header10= header_list[10]
    header11= header_list[11]
    header12= header_list[12]

    mainheader = "doc"

    output_string =(
    "<" + mainheader + ">\n" +
    "<field name=\"" + header0 + "\">" + element0 + "</field>\n" +
    "<field name=\"" + header1 + "\">" + element1 + "</field>\n" +
    "<field name=\"" + header2 + "\">" + element2 + "</field>\n" +
    "<field name=\"" + header3 + "\">" + element3 + "</field>\n" +
    "<field name=\"" + header4 + "\">" + element4 + "</field>\n" +
    "<field name=\""+ header5 + "\">" + element5 + "</field>\n" +
    "<field name=\""+ header6 + "\">" + element6 + "</field>\n" +
    "<field name=\"" + header7 + "\">" + element7 + "</field>\n" +
    "<field name=\"" + header8 + "\">" + element8 + "</field>\n" +
    "<field name=\"" + header9 + "\">" + element9 + "</field>\n" +
    "<field name=\"" + header10 + "\">" + element10 + "</field>\n" +
    "<field name=\"" + header11 + "\">" + element11 + "</field>\n" +
    "<field name=\"" + header12 + "\">" + element12 + "</field>\n" +
    "</" + mainheader + ">\n"
    )
    #print(output_string)

    return output_string

convert_row(data_list[1])

def convert_csv_to_xml(csv_as_list):
  """Convert the csv into XML as an output string
  """
  content_xml = ""
  for i in range(1, len(csv_as_list)): #Starts from 1 without header
    print(i)
    current_row_xml = convert_row(csv_as_list[i])
    content_xml = content_xml + (current_row_xml)
          
  output_string =(
  "<?xml version=\"1.0\"?> \n"+
  "<" + "add" + ">\n" +
  content_xml +
  "</" + "add" + ">\n"
  )

  #print(output_string)
  return output_string


outputstring = convert_csv_to_xml(data_list)

text_file = open("reviews.xml", "w")
text_file.write(outputstring)
text_file.close()
files.download('reviews.xml')